# Sample code for performing obstacle avoidance #

Import necessary packages.

In [ ]:
# Code adapted from: https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/autonomousSequence.py

import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

# CrazyFlie imports:

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander

Set your group number and camera number.

In [ ]:
group_number = 13

# Possibly try 0, 1, 2 ...
camera = 1

## Tune the red filtering ##

You can use the following cell to test and visualize the red filtering. This cell *not* make the drone fly. It will connect to the CrazyFlie camera and perform red filtering on the live video feed. You should use this cell to tune the HSV intervals, and then copy/paste your tuned intervals into the __check_contours__ function below. When tuning the intervals, keep in mind that the lighting in the environment can matter.

In [ ]:
def get_bounding_boxes(frame):
    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)
    
    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    boundRect = []
    for i, c in enumerate(contours):
        if cv2.contourArea(contours[i]) > 100:
            boundRect.append(cv2.boundingRect(cv2.approxPolyDP(c, 3, True)))
    
    return boundRect

def get_book_box(frame):
    lb1 = (5, 100, 100)
    ub1 = (30, 200, 200)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(hsv1, lb1, ub1)
    
    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    # Find the largest contour (should be the book)
    largest_area = 0
    largest_ind = -1
    
    for i, c in enumerate(contours):
        area = cv2.contourArea(c)
        if area > largest_area:
            largest_area = area
            largest_ind = i
    
    return cv2.boundingRect(cv2.approxPolyDP(contours[largest_ind], 3, True)) if largest_ind != -1 else None

In [ ]:
import random as rng
import numpy as np
import argparse
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    boundRect = get_bounding_boxes(frame)
    
    for i in range(len(boundRect)):
        color = (0, 255, 0)
        cv2.rectangle(frame, (int(boundRect[i][0]), int(boundRect[i][1])), \
          (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
    
    boundRect = get_book_box(frame)
    if boundRect is not None:
        color = (255,0,0)
        cv2.rectangle(frame, (int(boundRect[0]), int(boundRect[1])), \
          (int(boundRect[0]+boundRect[2]), int(boundRect[1]+boundRect[3])), color, 2)
    
    # shows all the bounding boxes for obstacles and the book
    cv2.imshow("Contours", frame)
    
    # Compute
    #cv2.imshow('mask', mask)    

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
# This block allows us to take screenshots and save the image
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    
    cv2.imshow("Contours", frame)
    key_pressed = cv2.waitKey(1)
    if key_pressed & 0xFF == ord('p'):
        cv2.imwrite("book.png", frame)
    
    # Hit q to quit.
    if key_pressed & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
# if code using the camera errors, then we call this block
cap.release()

In [ ]:
# if some opencv windows linger, we can call this to destroy them
cv2.destroyAllWindows()

In [ ]:
# test code for book bounding box
frame = cv2.imread('book.png')
boundRect, mask = get_book_box(frame)
print(boundRect)
if boundRect is not None:
    color = (255,0,0)
    cv2.rectangle(frame, (int(boundRect[0]), int(boundRect[1])), \
      (int(boundRect[0]+boundRect[2]), int(boundRect[1]+boundRect[3])), color, 2)
    
while True:
    cv2.imshow('mask', mask)
    cv2.imshow('frame', frame)

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

## Helper functions ##

The following cell contains some sample functions which will be useful.

In particular, __check_contours__ and __findGreatesContour__ will perform red filtering on the live camera feed and identify the obstacles. The red filtering is controlled by setting HSV intervals in the __check_contours__ function. Note that the intervals will require tuning and may vary on different drones/cameras.

The __adjust_position__ function can also be modified for performing obstacle avoidance.

In [ ]:
# Get the current crazyflie position:
def position_estimate(scf):
    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]
            x = data['kalman.varPX']
            y = data['kalman.varPY']
            z = data['kalman.varPZ']
            
    print(x, y, z)
    return x, y, z


# Set the built-in PID controller:
def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    time.sleep(2)
    return


# Ascend and hover at 1m:
def ascend_and_hover(cf):
    # Ascend:
    # NOTE: was changed from starter code - made ascent a little smoother
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 20)
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    return


# Sort through contours in the image
def findGreatesContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    #print(largest_area)

    return largest_area, largest_contour_index


# Find contours in the image
def check_contours(frame):

    print('Checking image:')

    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    largest_area, largest_contour_index = findGreatesContour(contours)

    print(largest_area)

    if largest_area > 100:
        return True
    else:
        return False


# Follow the setpoint sequence trajectory:
def adjust_position(cf, current_y):

    print('Adjusting position')

    steps_per_meter = int(10)
    # Set the number here (the iterations of the for-loop) to the number of side steps.
    # You may choose to tune the number and size of the steps.
    for i in range(3): 
        current_y = current_y - 1.0/float(steps_per_meter)
        position = [0, current_y, 0.5, 0.0]

        print('Setting position {}'.format(position))
        for i in range(10):
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)

    cf.commander.send_stop_setpoint()
    # Make sure that the last packet leaves before the link is closed.
    # The message queue is not flushed before closing.
    time.sleep(0.1)
    return current_y


# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    print('Descending:')
    # Hover at 0.5 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 25)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

## Test obstacle avoidance on the CrazyFlie ##

The following cell *will* fly the drone. Place the CrazyFlie in front of an obstacle in the netted area for testing. This cell will perform object detection and avoidance using the red filtering defined in the helper functions above.

In [ ]:
import time
# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

# Check that CrazyFlie devices are available:
if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascent to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf
        current_y = 0.0

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        cap = cv2.VideoCapture(camera)
        # CHANGED FROM STARTER CODE:
        # Currently, the drone hovers until we press q on the opencv window
        while(cap.isOpened()):

            ret, frame = cap.read()

            elapsed = time.time() - t
            if(elapsed > 5.0):
                if ret:
                    #cv2.imshow('frame',frame)

                    if(ascended_bool==0):
                        set_PID_controller(cf)
                        ascend_and_hover(cf)
                        ascended_bool = 1
                    else:
                        for _ in range(30):
                            cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
                            time.sleep(0.1)
                        boundRect = get_bounding_boxes(frame)
    
                        for i in range(len(boundRect)):
                            color = (0, 255, 0)
                            cv2.rectangle(frame, (int(boundRect[i][0]), int(boundRect[i][1])), \
                              (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
                        
                        boundRect = get_book_box(frame)
                        if boundRect is not None:
                            color = (255,0,0)
                            cv2.rectangle(frame, (int(boundRect[0]), int(boundRect[1])), \
                              (int(boundRect[0]+boundRect[2]), int(boundRect[1]+boundRect[3])), color, 2)
                        
                        cv2.imshow("Contours", frame)
                        # NOTE: DO NOT SPAM q - inputs are stored to a buffer and will cause subsequent trials to end early
                        if (cv2.waitKey(10) & 0xFF) == ord('q'):
                            print("hello")
                            break
        
        print(time.time() - t)
        cap.release()
        cv2.destroyAllWindows()

        # Descend and stop all motion:
        hover_and_descend(cf)

print('Done!')